In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datetime
import torch
import math

from modules.util.util import importstr
from modules.util.logconf import logging
log = logging.getLogger('nb')

In [3]:
def run(app, *argv):    
    torch.cuda.empty_cache()
    
    argv = list(argv)
    log.info("Running: {}({!r}).main()".format(app, argv))
    
    app_cls = importstr(*app.rsplit('.', 1))
    app_cls(argv).main()
    
    log.info("Finished: {}.{!r}.main()".format(app, argv))

In [4]:
# width_irc=(16,128,128)
# run('prepcache.CovidPrepCacheApp', f'--num-workers={8}', f'--batch-size={20}', '--width-irc', f'{width_irc[0]}', f'{width_irc[1]}', f'{width_irc[2]}')

2020-12-07 04:56:41,940 INFO     pid:22723 nb:005:run Running: prepcache.CovidPrepCacheApp(['--num-workers=8', '--batch-size=20', '--width-irc', '16', '128', '128']).main()
2020-12-07 04:56:44,332 INFO     pid:22723 prepcache:039:main Starting CovidPrepCacheApp, Namespace(batch_size=20, num_workers=8, width_irc=['16', '128', '128'])
100%|██████████| 10/10 [02:27<00:00, 14.72s/it]
2020-12-07 04:59:11,832 INFO     pid:22723 nb:010:run Finished: prepcache.CovidPrepCacheApp.['--num-workers=8', '--batch-size=20', '--width-irc', '16', '128', '128'].main()


## Basic Unet 3D

In [4]:
batch_size = 2
epochs = 50
steps_per_epoch=200
val_cadence = 5
recall_priority = 0
unet_depth=3
pad_types = ['zero', 'replicate']
windows = ['lung', 'mediastinal', 'shifted_lung']
project_name = 'covid19_seg'
width_irc = (12,192,192)

In [5]:
model_path = 'saved-models/2020-11-22_11.01.15.best.state'

In [ ]:
run_name = 'unet3d train run p2'

In [1]:
run('training.CovidSegmentationTrainingApp', f'--epochs={epochs}', f'--batch-size={batch_size}', 
    f'--steps-per-epoch={steps_per_epoch}', f'--val-cadence={val_cadence}', f'--recall-priority={recall_priority}',
    f'--depth={unet_depth}', f'--pad-type={pad_types[1]}', '--augmented', f'--model-path={model_path}',
    '--width-irc', f'{width_irc[0]}', f'{width_irc[1]}', f'{width_irc[2]}', 
    f'--project-name={project_name}', f'--run-name={run_name}')

# dummy run to finish wandb
run('training.CovidSegmentationTrainingApp', '--epochs=0', '--run-name=to delete')

## Covid Seg Net

In [5]:
batch_size = 2
epochs = 100
steps_per_epoch=320
val_cadence = 5
depth = 3
windows = ['lung', 'mediastinal', 'shifted_lung']
project_name = 'covid19_seg'
width_irc=(16,128,128)

In [6]:
run_name = 'Train p3 (320spe,2gpu,bs2x3,aug)'
notes = 'Train Run Continuation'

In [7]:
model_path = 'saved-models/2020-12-08_06.03.53.best.state'

In [8]:
model_path = 'saved-models/2020-12-08_06.03.53.best.state'

In [7]:
run('training_alpha.CovidSegmentationTrainingApp', f'--epochs={epochs}', f'--batch-size={batch_size}', f'--num-workers={8}',
    f'--ct-window={windows[2]}', f'--val-cadence={val_cadence}', f'--depth={depth}', '--augmented',
    '--width-irc', f'{width_irc[0]}', f'{width_irc[1]}', f'{width_irc[2]}', f'--notes={notes}',
    f'--model-path={model_path}',
    f'--steps-per-epoch={steps_per_epoch}', f'--project-name={project_name}', f'--run-name={run_name}')


# dummy run to finish wandb
run('training.CovidSegmentationTrainingApp', '--epochs=0', '--run-name=to delete')

2020-12-08 18:36:20,825 INFO     pid:6550 nb:005:run Running: training_alpha.CovidSegmentationTrainingApp(['--epochs=100', '--batch-size=2', '--num-workers=8', '--ct-window=shifted_lung', '--val-cadence=5', '--depth=3', '--augmented', '--width-irc', '16', '128', '128', '--notes=Train Run Continuation', '--model-path=saved-models/2020-12-08_06.03.53.best.state', '--steps-per-epoch=320', '--project-name=covid19_seg', '--run-name=Train p3 (320spe,2gpu,bs2x3,aug)']).main()


SyntaxError: invalid syntax (training_alpha.py, line 334)